## 作業
在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，並回答下列問題。
* Q1: 透過數值法計算  Age 和  Survived  是否有相關性?
* Q2:透過數值法計算  Sex 和 Survived  是否有相關性?
* Q3: 透過數值法計算  Age 和  Fare 是否有相關性?   
* 提示:    
1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
2.把題目中的 Survived 用 Survived_cate 來做分析       
3.首先觀察一下這些變數的資料型態後，再來想要以哪一種判斷倆倆的相關性。     


In [1]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display

import pingouin as pg
import researchpy   
%matplotlib inline

## 讀入資料

In [2]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [3]:
# 先清除遺失的欄位資料
# 列出遺失資料
#missing_index=df_train.isnull()
#display(df_train[missing_index.values==True])

In [4]:
# 刪除遺失的資料
#df_train_all = df_train.dropna()
#df_train_all

In [5]:
## 產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態
df_train['Survived_cate'] = df_train['Survived'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


###  Q1: 透過數值法計算  Age 和  Survived  是否有相關性?


In [6]:
# Age 為連續，Survived 為離散，因此使用 Eta Squared (η2)
data = df_train[['Age','Survived_cate']].dropna()
aov = pg.anova(dv='Age', between='Survived_cate', data=data, detailed=True)
aov

,Source,SS,DF,MS,F,p-unc,np2
0,Survived_cate,897.187582,1,897.187582,4.271195,0.039125,0.005963
1,Within,149559.448362,712,210.055405,NaN,NaN,NaN


In [7]:
etaSq = aov.SS[0] / (aov.SS[0] + aov.SS[1])
etaSq

0.0059630974469252

In [8]:
def judgment_etaSq(etaSq):
    if etaSq < .01:
        qual = 'Negligible'
    elif etaSq < .06:
        qual = 'Small'
    elif etaSq < .14:
        qual = 'Medium'
    else:
        qual = 'Large'
    return(qual)
print('Age 和 Survived 相關性：', judgment_etaSq(etaSq))

Age 和 Survived 相關性： Negligible


### Q2:透過數值法計算  Sex 和 Survived  是否有相關性?


In [9]:
# Sex 和 Survived 皆為 離散，因此使用 Cramer’s V 係數

# 刪除缺漏資料
data = df_train[['Sex','Survived_cate']].dropna()
#Step 1：用交叉列連表(contingency table) 整理資料
contTable = pd.crosstab(data['Sex'], data['Survived_cate'])
contTable

Survived_cate,0,1
Sex,,
female,81,233
male,468,109


In [10]:
#Step 2：計算資料自由度 df
df = min(contTable.shape[0], contTable.shape[1]) - 1
df

1

In [11]:
#Step 3：運用 researchpy 套件，計算出 Cramer’s V 係數
crosstab, res = researchpy.crosstab(data['Sex'], data['Survived_cate'], test='chi-square')
#print(res)
print("Cramer's value is",res.loc[2,'results'])

Cramer's value is 0.5434


In [12]:
## 寫一個副程式判斷相關性的強度
def judgment_CramerV(df,V):
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

In [13]:
print('Sex 和 Survived 相關性：', judgment_CramerV(df,res.loc[2,'results']))

Sex 和 Survived 相關性： large


### Q3: 透過數值法計算  Age 和  Fare 是否有相關性?  

In [14]:
def judgment_pearsonr(corr):
    if corr == 1:
        qual = '完全線性相關'
    if corr < .01:
        qual = '無線性相關'
    elif corr < .39:
        qual = '低度線性相關'
    elif corr < .69:
        qual = '中度線性相關'
    else:
        qual = '高度線性相關'
    return(qual)


In [15]:
# Age 和 Fare 皆為 連續，因此使用 Pearson 相關係數

# 刪除缺漏資料
data = df_train[['Age','Fare']].dropna()
# 由於 pearsonr 有兩個回傳結果，我們只需取第一個回傳值為相關係數
corr, _=stats.pearsonr(data['Age'], data['Fare'])
print('Age 和 Fare 相關性：', judgment_pearsonr(abs(corr)), '，值為：', corr)

Age 和 Fare 相關性： 低度線性相關 ，值為： 0.0960666917690389
